In [ ]:
# February 2025
# Data exploration and cleaning
# Violeta Berdejo-Espinola

# read data

In [ ]:
import mpu
from sklearn.preprocessing import LabelEncoder

# read classification model 

clf = mpu.io.read('../results/model/best_model.pickle')

# raw corpus

corpus_raw = mpu.io.read('../data/corpus_raw.pickle')

x = [' '.join(each_document) for each_document in corpus_raw]

# pos, negs

pos = mpu.io.read('../data/pos.pickle')
neg = mpu.io.read('../data/neg.pickle')
y = [1] * len(pos) + [0] * len(neg)

# shap of positive instances

In [ ]:
import shap
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split

# encode text to embeddings using SentenceTransformer
encoder = SentenceTransformer('distiluse-base-multilingual-cased-v1')
X = encoder.encode(x)

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=42,  test_size=0.20,)

# train a Logistic Regression model on embeddings
clf.fit(x_train, y_train)

# define SHAP text Explainer with token masking
masker = shap.maskers.Text() 

def prediction_function(text_batch):
    
    """Encodes perturbed text into embeddings and returns class probabilities."""
    
    embeddings = encoder.encode(text_batch)  # encode text to embeddings
    return clf.predict_proba(embeddings)[:]  # 1 to return probability for class 1

explainer = shap.Explainer(prediction_function, masker)

# compute SHAP Values
shap_values_all = explainer(x[:])

In [ ]:
# visualize SHAP feature importance of positive instances
shap.plots.text(shap_values_all[:63])

In [ ]:
# visualize SHAP feature importance of negative instances
shap.plots.text(shap_values_all[100:151])

# save shap.plot as html

In [ ]:
import matplotlib.pyplot as plt
from IPython.core.display import HTML

shap_values_positives = shap_values_all[:63] 

with open('../results/shap/shap_values_positive_instances.html', 'w') as file:
     file.write(shap.plots.text(shap_values_positives, display=False))

# features having the largest impact in positive predictions

In [ ]:
import matplotlib.pyplot as plt

shap_plot = shap.plots.bar(shap_values_positives.abs.max(0), show=False)

# plt.savefig("shap_plot_positive_instances.png",dpi=700, format="png", bbox_inches="tight")

plt.show()